In [1]:
# Package Installation Script

In [25]:
import pennylane as qml
from pennylane import numpy as np
import torch 
from torch.nn import MSELoss

In [26]:
dev = qml.device("default.qubit", wires=4)

In [33]:

def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.CNOT(wires=[0, 1])


@qml.qnode(dev, interface='torch')
def circuit(weights, data):
    qml.AmplitudeEmbedding(data, wires=range(2))
    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))


def variational_classifier(weights, bias, angles):
    return circuit(weights, angles) + bias



def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)
    return loss


In [34]:
class IrisDataset(torch.utils.data.Dataset):

    def __init__(self, X, labels):
        self.X = X
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        return self.X[idx], self.labels[idx]
    
    @property
    def data(self):
        return self.labels




In [35]:
# Data Loading and preprocessing

data = np.loadtxt("data/IrisData.txt")
X = data[:, 0:3]

# pad the vectors to size 2^2 with constant values
padding =  np.ones((len(X), 1))
X_pad = np.c_[X, padding]

# normalize each input
normalization = np.sqrt(np.sum(X_pad ** 2, -1))
features = (X_pad.T / normalization).T

Y = data[:, -1]

# Division of data into training and validation set
np.random.seed(0)
num_data = len(Y)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))

feats_train = features[index[:num_train]]
Y_train = Y[index[:num_train]]
train_dataloader = torch.utils.data.DataLoader(IrisDataset(feats_train, Y_train), batch_size=5, shuffle=True)

feats_val = torch.tensor(features[index[num_train:]], requires_grad=False)
Y_val = torch.tensor(Y[index[num_train:]], requires_grad=False)



In [36]:
epochs = 4
num_layers = 6
num_qubits = 2

weights_init = torch.tensor(0.01*np.random.rand(num_layers, num_qubits, 3), requires_grad=True)
bias_init = torch.tensor(0.0, requires_grad = True)
w = weights_init
b = bias_init

optim = torch.optim.Adam([w, b], lr=0.01)
loss_fn = MSELoss()

it=0
for ep in range(epochs):
    for X, labels in train_dataloader:
        pred = variational_classifier(w, b, X)
        optim.zero_grad()
        loss = loss_fn(pred, labels)
        loss.backward()
        optim.step()
        with torch.no_grad():
            pred_val = torch.sign(variational_classifier(w, b, feats_val))
            acc_val = accuracy(Y_val, pred_val)
        print(
            "Iter: {:5d} | Cost: {:0.7f} | Acc validation: {:0.7f} "
            "".format(it, loss.item(), acc_val)
        )
        it += 1


Iter:     0 | Cost: 1.8958831 | Acc validation: 0.4400000 
Iter:     1 | Cost: 1.2987891 | Acc validation: 0.4400000 
Iter:     2 | Cost: 1.8499143 | Acc validation: 0.4400000 
Iter:     3 | Cost: 1.4271211 | Acc validation: 0.4400000 
Iter:     4 | Cost: 1.5248606 | Acc validation: 0.3600000 
Iter:     5 | Cost: 2.2267096 | Acc validation: 0.3200000 
Iter:     6 | Cost: 1.1327880 | Acc validation: 0.2800000 
Iter:     7 | Cost: 1.3624653 | Acc validation: 0.2800000 
Iter:     8 | Cost: 1.5799411 | Acc validation: 0.2400000 
Iter:     9 | Cost: 1.1845444 | Acc validation: 0.2400000 
Iter:    10 | Cost: 1.1363760 | Acc validation: 0.2400000 
Iter:    11 | Cost: 1.4662111 | Acc validation: 0.2000000 
Iter:    12 | Cost: 1.0439381 | Acc validation: 0.2000000 
Iter:    13 | Cost: 1.5871979 | Acc validation: 0.2400000 
Iter:    14 | Cost: 1.7131773 | Acc validation: 0.2400000 
Iter:    15 | Cost: 1.3318863 | Acc validation: 0.2400000 
Iter:    16 | Cost: 1.6665369 | Acc validation: 0.320000